In [ ]:
import os
os.environ["OPENAI_API_KEY"]=""

In [ ]:
!pip install youtube-transcript-api langchain-community langchain-openai faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# Loading the data

In [ ]:
video_id = "VvqEennLlU0"
try:
   transcript_list =  YouTubeTranscriptApi().fetch(video_id=video_id, languages=["en"])
   transcript_raw = list(transcript_list.to_raw_data())
   transcript =  " ".join(item['text']for item in transcript_raw)
   print(transcript)
except TranscriptsDisabled:
    print("No captions available for this video")

The fate of all tech companies will now be decided by their AI capabilities. Google CEO thinks that AI is >> the most profound technology humanity is working on, more profound than fire or electricity or anything that we have done in the past. >> Microsoft CEO said that with AI, we are going to go to the next level of abstraction. Zuckerberg's even spending billions poaching the best AI researchers to work on their own AI replacements. We are trying to build a coding agent that basically advances llama research specifically. I would guess that like sometime in the next 12 to 18 months, we'll reach the point where like most of the code that's going towards these efforts is written by AI. It's clear that big tech is having a competition over whose language model is the largest because the race to AGI will define everything in the future. They're so sure of this that these CEOs are now spending hundreds of billions building data centers and poaching top researchers all just to get a leg u

In [ ]:
print(type(transcript))

<class 'str'>


Text splitting

In [ ]:
splitter =  RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks =  splitter.create_documents([transcript])

In [ ]:
type(chunks)

list

In [ ]:
print(chunks)

[Document(metadata={}, page_content="The fate of all tech companies will now be decided by their AI capabilities. Google CEO thinks that AI is the most profound technology humanity is working on, more profound than fire or electricity or anything that we have done in the past. Microsoft CEO said that with AI, we are going to go to the next level of abstraction. Zuckerberg's even spending billions poaching the best AI researchers to work on their own AI replacements. We are trying to build a coding agent that basically advances llama research specifically. I would guess that like sometime in the next 12 to 18 months, we'll reach the point where like most of the code that's going towards these efforts is written by AI. It's clear that big tech is having a competition over whose language model is the largest because the race to AGI will define everything in the future. They're so sure of this that these CEOs are now spending hundreds of billions building data centers and poaching top rese

In [ ]:
len(chunks)

30

## Embedding in Vector store

In [ ]:
embeddings =  OpenAIEmbeddings(model="text-embedding-3-small")
vector_store =  FAISS.from_documents(chunks,embedding=embeddings)

In [ ]:
vector_store.index_to_docstore_id

{0: 'ca18d3fe-826c-4fa5-9bfe-b6214b714f32',
 1: '8a0c4b2a-7118-49bb-92c2-e08da5e7d71c',
 2: '7ed4de7e-135a-4d45-9623-d448f8e90232',
 3: 'e50f9913-2eae-40eb-ab56-871b5d37d729',
 4: '6c8be239-9354-479e-a4ed-d12d124c3ab4',
 5: '896ccc15-a9cc-429d-8b5d-e97e9d3dc447',
 6: 'b5fd22b9-167e-472f-a189-3caca2516266',
 7: '649fddbc-6567-4339-93eb-1701005ee5a8',
 8: '6443c63f-3f9d-4680-b4f7-69a204164cc4',
 9: '3d94e650-ec6d-4a69-827d-78b24a0fb7e8',
 10: 'eb092dbf-74f0-475c-ad38-5ca364a64c5c',
 11: '6818c613-7988-420b-a45d-0fde175258bf',
 12: '812fb567-3960-46f0-9695-6f16dbd87970',
 13: '18e91205-6739-4f96-bfcf-15f3a684a746',
 14: 'c3d8ce13-ba5a-4ae2-868d-45c3bfe0bf2b',
 15: '63b00841-7b9a-4915-b6f4-eb25786274aa',
 16: 'fa5b25f6-17b5-47f2-bb0d-d9c21173c468',
 17: 'da967de0-652f-40ac-936d-351600e39d8c',
 18: 'b719feda-1aed-40dd-91b6-54772fa16b66',
 19: 'e5ff1756-7339-4bc8-bbca-b813c92e855a',
 20: '86c8aa60-8c45-41dc-a8ea-ce9e7063994a',
 21: '836ff5cb-f9fc-4555-bbdd-91561ffb81b2',
 22: 'a9667aab-2a2c-

In [ ]:
vector_store.get_by_ids(["ca18d3fe-826c-4fa5-9bfe-b6214b714f32"])

[Document(id='ca18d3fe-826c-4fa5-9bfe-b6214b714f32', metadata={}, page_content="The fate of all tech companies will now be decided by their AI capabilities. Google CEO thinks that AI is the most profound technology humanity is working on, more profound than fire or electricity or anything that we have done in the past. Microsoft CEO said that with AI, we are going to go to the next level of abstraction. Zuckerberg's even spending billions poaching the best AI researchers to work on their own AI replacements. We are trying to build a coding agent that basically advances llama research specifically. I would guess that like sometime in the next 12 to 18 months, we'll reach the point where like most of the code that's going towards these efforts is written by AI. It's clear that big tech is having a competition over whose language model is the largest because the race to AGI will define everything in the future. They're so sure of this that these CEOs are now spending hundreds of billions 

## Retrieval

In [ ]:
retriever =  vector_store.as_retriever(search_type="mmr",search_kwargs={"k" : 2})

In [ ]:
retriever.invoke("what is the meaning of life")

[Document(id='b5fd22b9-167e-472f-a189-3caca2516266', metadata={}, page_content="someone sent you without scrolling back through the messages? Siri had the answer. Need to reach into your phone and find some old documents or details from years ago? Siri would know how to navigate through it all, down to the names of people in your photos. New contextual awareness meant that Siri was going to be like an actual person inside of your phone who would understand any question you asked it. Even if it didn't have the answer, it could just pass it on to Chat GBT instead. all securely and without sharing your data. Of course, people were hyped up. Apple had promised their loyal fans the world. They weren't going to let them down, right? Apple even made it the biggest selling point of their next line of phones. Only the previous years as pros models in the iPhone 16 series would have Apple intelligence, and Apple's built an entire brand on trust and loyalty. And the best part was fans didn't have

## Augmentation

In [ ]:
llm =  ChatOpenAI(model="gpt-4o-mini",temperature=0.2)

In [ ]:
prompt =  PromptTemplate(
    template= """
      You are a helpful assistant
      Answer ONLY from provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      question: {question}
    """,
    input_variables=['context', 'question']
)

In [ ]:
question          = "what did the google ceo say about ai"
retrieved_docs    = retriever.invoke(question)

In [ ]:
retrieved_docs

[Document(id='ca18d3fe-826c-4fa5-9bfe-b6214b714f32', metadata={}, page_content="The fate of all tech companies will now be decided by their AI capabilities. Google CEO thinks that AI is the most profound technology humanity is working on, more profound than fire or electricity or anything that we have done in the past. Microsoft CEO said that with AI, we are going to go to the next level of abstraction. Zuckerberg's even spending billions poaching the best AI researchers to work on their own AI replacements. We are trying to build a coding agent that basically advances llama research specifically. I would guess that like sometime in the next 12 to 18 months, we'll reach the point where like most of the code that's going towards these efforts is written by AI. It's clear that big tech is having a competition over whose language model is the largest because the race to AGI will define everything in the future. They're so sure of this that these CEOs are now spending hundreds of billions 

In [ ]:
contetxt = "\n|n".join(doc.page_content for doc in retrieved_docs)

In [ ]:
final_prompt =  prompt.invoke({'context':contetxt, 'question': question})

In [ ]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant\n      Answer ONLY from provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      The fate of all tech companies will now be decided by their AI capabilities. Google CEO thinks that AI is the most profound technology humanity is working on, more profound than fire or electricity or anything that we have done in the past. Microsoft CEO said that with AI, we are going to go to the next level of abstraction. Zuckerberg's even spending billions poaching the best AI researchers to work on their own AI replacements. We are trying to build a coding agent that basically advances llama research specifically. I would guess that like sometime in the next 12 to 18 months, we'll reach the point where like most of the code that's going towards these efforts is written by AI. It's clear that big tech is having a competition over whose language model is the largest because the race to AGI will defin

In [ ]:
ans =  llm.invoke(final_prompt)
print(ans)

content='The Google CEO thinks that AI is the most profound technology humanity is working on, more profound than fire or electricity or anything that we have done in the past.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 441, 'total_tokens': 473, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C4hH5WdwTYWrkMbPkLi0ZeKKunM4W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--01be135b-2c4b-48c6-a05a-86b6e8709669-0' usage_metadata={'input_tokens': 441, 'output_tokens': 32, 'total_tokens': 473, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## **Using chains**

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n|n".join(docs.page_content for docs in retrieved_docs)
  return context_text

chain-1 for adding the context

In [ ]:
parallel_chain  =  RunnableParallel(
    {'context' : retriever | RunnableLambda(format_docs),
    'question' : RunnablePassthrough()}
)

In [ ]:
parser =  StrOutputParser()

In [ ]:
chain =  parallel_chain | prompt | llm | parser

In [ ]:
chain.invoke("Summarize the video")

"The video discusses Apple's brand built on trust and loyalty, highlighting the quick release of new technology following a June conference. It introduces video AI as a revolutionary tool for video creation, allowing users to generate full-length videos easily with just a text prompt, eliminating the need for cameras or editing. The video also critiques Tim Cook's leadership at Apple, suggesting he has lost control over the company and its vision, particularly in light of recent product failures like the Vision Pro goggles, which are seen as overpriced and impractical for average consumers."